In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/docxfiles/Climate_policy.docx
/kaggle/input/docxfiles/Baggage Guidelines.pdf
/kaggle/input/docxfiles/AI_healthcare.docx


In [2]:
!pip install -q docx2txt


In [3]:
!pip install -q langchain langchain-community sentence-transformers chromadb pypdf python-docx pandas transformers rank_bm25


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 64.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 55.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 72.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━

In [4]:
import os
import numpy as np
from pathlib import Path

# Document loaders
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Embeddings + Vector DB
from sentence_transformers import SentenceTransformer
import chromadb

# BM25
from rank_bm25 import BM25Okapi

# Cross-encoder reranker
from sentence_transformers import CrossEncoder

# Summarization / QA
from transformers import pipeline


2026-02-12 22:31:54.836604: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770935515.058935      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770935515.128099      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770935515.685077      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770935515.685122      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770935515.685124      55 computation_placer.cc:177] computation placer alr

In [5]:
def load_documents(folder_path):
    all_docs = []
    
    for file in os.listdir(folder_path):
        path = os.path.join(folder_path, file)
        
        if file.endswith(".pdf"):
            loader = PyPDFLoader(path)
            docs = loader.load()
        elif file.endswith(".docx"):
            loader = Docx2txtLoader(path)
            docs = loader.load()
        elif file.endswith(".csv"):
            loader = CSVLoader(path)
            docs = loader.load()
        else:
            continue
        
        for d in docs:
            d.metadata["source"] = file
        
        all_docs.extend(docs)
    
    return all_docs


In [6]:
def split_documents(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=50
    )
    return splitter.split_documents(documents)


In [7]:
class SemanticIndex:
    def __init__(self, persist_dir="db"):
        self.model = SentenceTransformer("all-MiniLM-L6-v2")
        self.client = chromadb.Client()
        self.collection = self.client.create_collection("docs")
    
    def build(self, docs):
        texts = [d.page_content for d in docs]
        metadatas = [d.metadata for d in docs]
        
        embeddings = self.model.encode(texts, show_progress_bar=True)
        
        ids = [str(i) for i in range(len(texts))]
        
        self.collection.add(
            documents=texts,
            embeddings=embeddings,
            metadatas=metadatas,
            ids=ids
        )
    
    def search(self, query, k=5):
        q_emb = self.model.encode([query])[0]
        
        results = self.collection.query(
            query_embeddings=[q_emb],
            n_results=k
        )
        
        docs = []
        for i in range(len(results["documents"][0])):
            docs.append({
                "text": results["documents"][0][i],
                "metadata": results["metadatas"][0][i]
            })
        
        return docs


In [8]:
class BM25Index:
    def build(self, docs):
        self.texts = [d.page_content for d in docs]
        self.metadatas = [d.metadata for d in docs]
        tokenized = [t.split() for t in self.texts]
        self.bm25 = BM25Okapi(tokenized)
    
    def search(self, query, k=5):
        scores = self.bm25.get_scores(query.split())
        top_k = np.argsort(scores)[-k:][::-1]
        
        results = []
        for idx in top_k:
            results.append({
                "text": self.texts[idx],
                "metadata": self.metadatas[idx]
            })
        
        return results


In [9]:
class Reranker:
    def __init__(self):
        self.model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
    
    def rerank(self, query, docs, top_k=3):
        pairs = [(query, d["text"]) for d in docs]
        scores = self.model.predict(pairs)
        
        ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
        return [r[0] for r in ranked[:top_k]]


In [10]:
class HybridRetriever:
    def __init__(self, semantic_index, bm25_index, reranker):
        self.semantic = semantic_index
        self.bm25 = bm25_index
        self.reranker = reranker
    
    def retrieve(self, query, k=5):
        sem = self.semantic.search(query, k)
        kw = self.bm25.search(query, k)
        
        combined = sem + kw
        
        # Remove duplicates
        unique = []
        seen = set()
        for d in combined:
            if d["text"] not in seen:
                unique.append(d)
                seen.add(d["text"])
        
        return self.reranker.rerank(query, unique, top_k=k)
        # return unique[:3]


In [11]:
qa_model = pipeline("text2text-generation", model="google/flan-t5-base")

def answer_question(query, docs):
    context = "\n".join([d["text"] for d in docs])
    
    prompt = f"""
    Answer based only on the context.

    Context:
    {context}

    Question: {query}
    """
    
    response = qa_model(prompt, max_length=200)[0]["generated_text"]
    
    sources = set([d["metadata"]["source"] for d in docs])
    
    return response, sources


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [12]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_documents(docs):
    text = " ".join([d.page_content for d in docs[:20]])
    summary = summarizer(text, max_length=200, min_length=50)[0]["summary_text"]
    return summary


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [13]:
def recall_at_k(retrieved_docs, true_source):
    sources = [d["metadata"]["source"] for d in retrieved_docs]
    return int(true_source in sources)


In [14]:
def evaluate(query, true_source, semantic, bm25, hybrid):
    sem_docs = semantic.search(query, k=3)
    bm_docs = bm25.search(query, k=3)
    hy_docs = hybrid.retrieve(query)
    
    return {
        "Semantic": recall_at_k(sem_docs, true_source),
        "BM25": recall_at_k(bm_docs, true_source),
        "Hybrid": recall_at_k(hy_docs, true_source)
    }


In [15]:
docs = load_documents("/kaggle/input/docxfiles")
chunks = split_documents(docs)

semantic = SemanticIndex()
semantic.build(chunks)

bm25 = BM25Index()
bm25.build(chunks)

reranker = Reranker()
hybrid = HybridRetriever(semantic, bm25, reranker)




modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [16]:
from collections import Counter

def retrieval_report(retrieved_docs):
    """
    Shows how many chunks came from each source document
    """
    sources = [d["metadata"]["source"] for d in retrieved_docs]
    count = Counter(sources)
    
    print("\n===== RETRIEVAL REPORT =====")
    total = len(retrieved_docs)
    
    for src, c in count.items():
        percent = (c / total) * 100
        print(f"{src}: {c} chunks ({percent:.1f}%)")
    
    print("Total retrieved chunks:", total)

from collections import Counter

def relevance_check(results, min_support=2):
    """
    Returns True if enough chunks support the answer.
    Prevents hallucination when retrieval is weak.
    """
    if len(results) == 0:
        return False
    
    sources = [d["metadata"]["source"] for d in results]
    count = Counter(sources)
    top_count = count.most_common(1)[0][1]
    
    return top_count >= min_support

def format_answer(answer, sources):
    print("\n===== ANSWER =====")
    print(answer)
    
    print("\n===== SOURCES =====")
    for s in sources:
        print("-", s)

def limit_per_source(docs, max_per_source=3):
    result = []
    count = {}
    
    for d in docs:
        src = d["metadata"]["source"]
        if count.get(src, 0) < max_per_source:
            result.append(d)
            count[src] = count.get(src, 0) + 1
    
    return result


In [17]:
query = "What is the restricted items during air travel"

results = hybrid.retrieve(query)
results = limit_per_source(results, max_per_source=3)

for r in results:
    print(r["metadata"]["source"])
retrieval_report(results)

if not relevance_check(results):
    print("\n===== ANSWER =====")
    print("Information not found in the documents.")
else:
    answer, sources = answer_question(query, results)
    format_answer(answer, sources)

print("Answer:", answer)
print("Sources:", sources)

Baggage Guidelines.pdf
Baggage Guidelines.pdf
Baggage Guidelines.pdf

===== RETRIEVAL REPORT =====
Baggage Guidelines.pdf: 3 chunks (100.0%)
Total retrieved chunks: 3


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



===== ANSWER =====
E-cigarettes Lighter Fluid Power Banks Spare Battery Safety Matches Laptop/ Tab Dry Coconut/ Copra Smart Suitcase POWER BANK Power bank, batteries, spare/loose, including lithium metal or lithium ion cells, for portable electronic devices must be carried in cabin baggage only

===== SOURCES =====
- Baggage Guidelines.pdf
Answer: E-cigarettes Lighter Fluid Power Banks Spare Battery Safety Matches Laptop/ Tab Dry Coconut/ Copra Smart Suitcase POWER BANK Power bank, batteries, spare/loose, including lithium metal or lithium ion cells, for portable electronic devices must be carried in cabin baggage only
Sources: {'Baggage Guidelines.pdf'}


In [18]:
from collections import Counter

chunk_sources = [c.metadata["source"] for c in chunks]
print(Counter(chunk_sources))


Counter({'Baggage Guidelines.pdf': 6, 'Climate_policy.docx': 3, 'AI_healthcare.docx': 3})


In [19]:
test_set = [
    ("What will AI do in healthcare?", "AI_healthcare.docx"),
    ("Explain restricted items in airport", "Baggage Guidelines.pdf"),
    ("What policies are there to protect the climate?", "Climate_policy.docx"),
]


In [20]:
def recall_at_k(retrieved_docs, true_source):
    sources = [d["metadata"]["source"] for d in retrieved_docs]
    return int(true_source in sources)


In [21]:
def evaluate_methods(test_set, semantic, bm25, hybrid):
    results = {
        "Semantic": 0,
        "BM25": 0,
        "Hybrid": 0
    }
    
    for query, true_src in test_set:
        sem_docs = semantic.search(query, k=3)
        bm_docs = bm25.search(query, k=3)
        hy_docs = hybrid.retrieve(query)
        
        results["Semantic"] += recall_at_k(sem_docs, true_src)
        results["BM25"] += recall_at_k(bm_docs, true_src)
        results["Hybrid"] += recall_at_k(hy_docs, true_src)
    
    total = len(test_set)
    
    print("\n===== EVALUATION (Recall@3) =====")
    for method in results:
        score = results[method] / total
        print(f"{method}: {score:.2f}")


In [22]:
evaluate_methods(test_set, semantic, bm25, hybrid)



===== EVALUATION (Recall@3) =====
Semantic: 1.00
BM25: 0.33
Hybrid: 1.00
